# i. Perkenalan
- Nama: Stanly
- Batch: RMT-036
- Tujuan: Untuk membuat Logistic Regression machine learning model, untuk memprediksi apakah seorang customer akan membeli produk saat kembali mengunjungi website. Data yang diambil adalah dari Google BigQuery `data-to-insights.ecommerce.web_analytics`

# ii. Query SQL

dataset_1:

In [514]:
"""
dataset_1 = client.query('''
SELECT * EXCEPT(fullVisitorId)

FROM
  # features
  (SELECT
    fullVisitorId,
    IFNULL(totals.bounces, 0) AS bounces,
    IFNULL(totals.timeOnSite, 0) AS time_on_site
  FROM
    `data-to-insights.ecommerce.web_analytics`
  WHERE
    totals.newVisits = 1 #gunakan hanya untuk pengunjung baru
    AND date BETWEEN '20160801' AND '20170531') # latih hanya pada 10 bulan pertama
  JOIN
  (SELECT
    fullvisitorid,
    # membuat label atau target
    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM
      `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid)
  USING (fullVisitorId)
  ;
''').to_dataframe()

print(len(dataset_1))
dataset_1.to_csv("dataset_1.csv")
dataset_1.head()
"""

'\ndataset_1 = client.query(\'\'\'\nSELECT * EXCEPT(fullVisitorId)\n\nFROM\n  # features\n  (SELECT\n    fullVisitorId,\n    IFNULL(totals.bounces, 0) AS bounces,\n    IFNULL(totals.timeOnSite, 0) AS time_on_site\n  FROM\n    `data-to-insights.ecommerce.web_analytics`\n  WHERE\n    totals.newVisits = 1 #gunakan hanya untuk pengunjung baru\n    AND date BETWEEN \'20160801\' AND \'20170531\') # latih hanya pada 10 bulan pertama\n  JOIN\n  (SELECT\n    fullvisitorid,\n    # membuat label atau target\n    IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit\n  FROM\n      `data-to-insights.ecommerce.web_analytics`\n  GROUP BY fullvisitorid)\n  USING (fullVisitorId)\n  ;\n\'\'\').to_dataframe()\n\nprint(len(dataset_1))\ndataset_1.to_csv("dataset_1.csv")\ndataset_1.head()\n'

dataset_2:

In [515]:
"""
dataset_2 = client.query('''
WITH all_visitor_stats AS (
SELECT
  fullvisitorid,
  #1. buatlah label atau target  "will_buy_on_return_visit" dan hilangkan komentar pada code tersebut .
  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit
  FROM `data-to-insights.ecommerce.web_analytics`
  GROUP BY fullvisitorid
)

SELECT * EXCEPT(unique_session_id) FROM (

  SELECT
      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,
      will_buy_on_return_visit,

      # 2. lengkapi code berikut ini dengan ketentuan ubahlah type datanya menjadi integer dan hilangkan komentar pada code tersebut.
      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,

      # perilaku pengguna di situs web
      IFNULL(totals.bounces, 0) AS bounces,
      IFNULL(totals.timeOnSite, 0) AS time_on_site,
      totals.pageviews,

      # informasi dari mana pengunjung itu berasal
      trafficSource.source,
      trafficSource.medium,
      channelGrouping,

      # informasi tentang device yang digunakan user
      device.deviceCategory,

      # informasi tentang geografi user
      IFNULL(geoNetwork.country, "Unknown") AS country

  FROM `data-to-insights.ecommerce.web_analytics`,
    UNNEST(hits) AS h

    JOIN all_visitor_stats USING(fullvisitorid)

  WHERE
    totals.newVisits = 1
    # 3. Gunakan data dari tanggal '20160801' hingga '20170531'
    AND date BETWEEN '20160801' AND '20170531'
  GROUP BY
  unique_session_id,
  will_buy_on_return_visit,
  bounces,
  time_on_site,
  totals.pageviews,
  trafficSource.source,
  trafficSource.medium,
  channelGrouping,
  device.deviceCategory,
  country
)
''').to_dataframe()
print(len(dataset_2))
dataset_2.to_csv("dataset_2.csv")
dataset_2.head()
"""

'\ndataset_2 = client.query(\'\'\'\nWITH all_visitor_stats AS (\nSELECT\n  fullvisitorid,\n  #1. buatlah label atau target  "will_buy_on_return_visit" dan hilangkan komentar pada code tersebut .\n  IF(COUNTIF(totals.transactions > 0 AND totals.newVisits IS NULL) > 0, 1, 0) AS will_buy_on_return_visit\n  FROM `data-to-insights.ecommerce.web_analytics`\n  GROUP BY fullvisitorid\n)\n\nSELECT * EXCEPT(unique_session_id) FROM (\n\n  SELECT\n      CONCAT(fullvisitorid, CAST(visitId AS STRING)) AS unique_session_id,\n      will_buy_on_return_visit,\n\n      # 2. lengkapi code berikut ini dengan ketentuan ubahlah type datanya menjadi integer dan hilangkan komentar pada code tersebut.\n      MAX(CAST(h.eCommerceAction.action_type AS INT64)) AS latest_ecommerce_progress,\n\n      # perilaku pengguna di situs web\n      IFNULL(totals.bounces, 0) AS bounces,\n      IFNULL(totals.timeOnSite, 0) AS time_on_site,\n      totals.pageviews,\n\n      # informasi dari mana pengunjung itu berasal\n      tr

# iii. Pendahuluan

Masalah yang ingin dipecahkan pada dataset ini adalah untuk memprediksi apakah pengunjung akan melakukan pembelian produk ketika dia mengunjungi website lagi di lain waktu. Data-data yang bisa menjadi penunjang prediksi meliputi: status terakhir pada saat mengunjungi website, web analytics bounce rate, berapa lama waktu mengunjungi website, berapa halaman yang dilihat, sumber website, medium, channel grouping, device yang dipakai dan negara. Dengan menganalisa data-data penunjang tersebut, kita bisa memprediksi apakah pengunjung akan membeli produk di lain waktu.

Dengan menganalisa data-data tersebut, kita juga bisa menganalisa, apakah data bounces, time_on_site dan pageviews memiliki pengaruh terhadap minat pembelian user?

# iv. Import Libraries

In [516]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
import pickle

# v. Data Loading

In [517]:
dataset_1 = pd.read_csv("dataset_1.csv", index_col=0)
dataset_2 = pd.read_csv("dataset_2.csv", index_col=0)

# vi. Exploratory Data Analysis (EDA)

## i. Apakah fitur bounces, time_on_site, dan pageviews memiliki pengaruh yang cukup tinggi untuk memprediksi target will_buy_on_return_visit?

In [518]:
columns = ["bounces","time_on_site","pageviews"]
for col in columns:
    print(f'The columns {col} correlation with the target is: {dataset_2[col].corr(dataset_2["will_buy_on_return_visit"])}')

The columns bounces correlation with the target is: -0.10166587704535601
The columns time_on_site correlation with the target is: 0.12185463760670473
The columns pageviews correlation with the target is: 0.15433059441989552


Dari hasil korelasi di atas, bisa kita simpulkan bahwa kolom `bounces`, `time_on_site` dan `pageviews` memiliki korelasi yang lemah terhadap pengaruh pembelian produk oleh pengunjung di lain waktu.

## ii. Apa jenis device yang paling sering digunakan oleh user di wilayah Canada yang berpotensi melakukan pembelian saat mengunjungi kembali website tersebut ?

In [519]:
dataset_2[dataset_2["country"] == "Canada"].groupby("deviceCategory")["will_buy_on_return_visit"].value_counts().to_frame()

count
deviceCategory will_buy_on_return_visit       
desktop        0                         11031
               1                           189
mobile         0                          3979
               1                            11
tablet         0                           899
               1                             1

Dari dataframe di atas, bisa kita lihat bahwa device yang paling sering digunakan oleh user di wilayah Canada, yang berpotensi melakukan pembelian saat mengunjungi kembali website tersebut adalah `desktop`.

# vii. Modeling 1 - dataset_1.csv

## i. Feature Engineering

Check for missing values

In [520]:
dataset_1.isna().sum()

bounces                     0
time_on_site                0
will_buy_on_return_visit    0
dtype: int64

In [521]:
X_1 = dataset_1.drop(['will_buy_on_return_visit'], axis=1)
y_1 = dataset_1['will_buy_on_return_visit']
X_1.head()

,bounces,time_on_site
0,0,733
1,1,0
2,0,45
3,0,951
4,0,389


In [522]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.25, random_state=10)

In [523]:
X_train_1.head()

,bounces,time_on_site
211952,1,0
79567,0,125
52849,0,62
58632,1,0
231633,1,0


Cek skewness di setiap kolom

In [524]:
train_columns_1 = X_train_1.columns.to_list()
for the_col in train_columns_1:
  print(f'{the_col} has skewness of: {X_train_1[the_col].skew()}')

bounces has skewness of: -0.006162442166215095
time_on_site has skewness of: 6.245061102386208


time_on_site memiliki skewness yang tinggi, maka akan kita capping menggunakan Winsorizer, dengan metode `iqr`. Karena nilainya yang sangat tinggi, argumen fold kita beri `3`

In [525]:
winsorizer_1 = Winsorizer(capping_method='iqr',
                             tail='both',
                             fold=3,
                             variables=['time_on_site'])

X_train_1 = winsorizer_1.fit_transform(X_train_1)
X_test_1 = winsorizer_1.transform(X_test_1)

Skewness sekarang sudah berkurang

In [526]:
train_columns_1 = X_train_1.columns.to_list()
for the_col in train_columns_1:
  print(f'{the_col} has skewness of: {X_train_1[the_col].skew()}')

bounces has skewness of: -0.006162442166215095
time_on_site has skewness of: 1.5980581869308457


In [527]:
scaler_1 = MinMaxScaler()
encoder_1 = OneHotEncoder()

Menggunakan pipeline

In [528]:
num_pipe_1 = Pipeline([('scaling', scaler_1)])
cat_pipe_1 = Pipeline([('encoding', encoder_1)])
num_pipe_1

Pipeline(steps=[('scaling', MinMaxScaler())])

In [529]:
transformer_1 = ColumnTransformer([('categorical', cat_pipe_1, ["bounces"]),
                                 ('numerical', num_pipe_1, ["time_on_site"])])
transformer_1

ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('encoding',
                                                  OneHotEncoder())]),
                                 ['bounces']),
                                ('numerical',
                                 Pipeline(steps=[('scaling', MinMaxScaler())]),
                                 ['time_on_site'])])

In [530]:
transformer_1.fit(X_train_1)
X_train_final_1 = transformer_1.transform(X_train_1)
X_test_final_1 = transformer_1.transform(X_test_1)

In [531]:
X_train_final_1[:5]

array([[0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.38580247],
       [1.        , 0.        , 0.19135802],
       [0.        , 1.        , 0.        ],
       [0.        , 1.        , 0.        ]])

## ii. Model Definition

Menggunakan Logistic Regression, karena model machine learning ini sangat bagus untuk memprediksi kasus klasifikasi.

In [532]:
model_logreg_1 = LogisticRegression(penalty='l2', C=1.0, random_state=0)

In [533]:
lr_pipe_1 = Pipeline([('feature_engineering', transformer_1),
                    ('lr', model_logreg_1)])

lr_pipe_1

Pipeline(steps=[('feature_engineering',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoding',
                                                                   OneHotEncoder())]),
                                                  ['bounces']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaling',
                                                                   MinMaxScaler())]),
                                                  ['time_on_site'])])),
                ('lr', LogisticRegression(random_state=0))])

## iii. Model Training

In [534]:
lr_pipe_1.fit(X_train_1, y_train_1)
y_pred_train_1 = lr_pipe_1.predict(X_train_1)
y_pred_test_1 = lr_pipe_1.predict(X_test_1)
y_pred_train_1

array([0, 0, 0, ..., 0, 0, 0])

In [535]:
def check_roc_auc_score(auc_score):
  if auc_score > 0.9:
    return "Good"
  elif auc_score <= 0.9 and auc_score >= 0.8:
    return "Fair"
  elif auc_score < 0.8 and auc_score >= 0.7:
    return "Decent"
  elif auc_score < 0.7 and auc_score >= 0.6:
    return "Not Great"
  else:
    return "Poor"

## iv. Model Evaluation

In [536]:
y_pred_proba_lr_pipe_1 = lr_pipe_1.predict_proba(X_test_1)[:, 1]
auc_logreg_1 = roc_auc_score(y_test_1, y_pred_proba_lr_pipe_1)

print('AUC Score:', auc_logreg_1)
print('AUC Category:', check_roc_auc_score(auc_logreg_1))

AUC Score: 0.7766557738985467
AUC Category: Decent


In [537]:
y_pred_proba_lr_pipe_1 = lr_pipe_1.predict_proba(X_train_1)[:, 1]
auc_logreg_1 = roc_auc_score(y_train_1, y_pred_proba_lr_pipe_1)

print('AUC Score:', auc_logreg_1)
print('AUC Category:', check_roc_auc_score(auc_logreg_1))

AUC Score: 0.7808722178728786
AUC Category: Decent


2 AUC Score menunjukkan hasil yang cukup baik (X_test terhadap y_test dan X_train terhadap y_train)

In [538]:
print('Error - Train set:', mean_absolute_error(y_train_1, y_pred_train_1))
print('Error - Test set:', mean_absolute_error(y_test_1, y_pred_test_1))

Error - Train set: 0.0155
Error - Test set: 0.01533964525868424


Dapat disimpulkan, dengan nilai training dan test error yang sangat rendah berarti model ini cukup baik untuk memprediksi data yang sudah diberikan.

# viii. Modeling 2 - dataset_2.csv

## i. Feature Engineering

Splitting features dan target

In [539]:
X_2 = dataset_2.drop(['will_buy_on_return_visit'], axis=1)
y_2 = dataset_2['will_buy_on_return_visit']
X_2.head()

,latest_ecommerce_progress,bounces,time_on_site,pageviews,source,medium,channelGrouping,deviceCategory,country
0,0,0,539,5.0,google,organic,Organic Search,desktop,Germany
1,0,0,72,5.0,google,organic,Organic Search,tablet,United States
2,0,0,823,5.0,google,organic,Organic Search,desktop,Turkey
3,0,0,160,5.0,google,organic,Organic Search,desktop,United States
4,0,0,547,5.0,google,organic,Organic Search,desktop,Italy


Check for missing values

In [540]:
X_2.isna().sum()

latest_ecommerce_progress     0
bounces                       0
time_on_site                  0
pageviews                    14
source                        0
medium                        0
channelGrouping               0
deviceCategory                0
country                       0
dtype: int64

Train-test splitting

In [541]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.25, random_state=10)

In [542]:
cat_columns_2 = ["latest_ecommerce_progress", "bounces", "source", "medium", "channelGrouping", "deviceCategory", "country"]
num_columns_2 = ["time_on_site", "pageviews"]

In [543]:
for cat_col in cat_columns_2:
  print(f'Total unique values of column {cat_col}: {len(X_train_2[cat_col].unique())}')

Total unique values of column latest_ecommerce_progress: 7
Total unique values of column bounces: 2
Total unique values of column source: 261
Total unique values of column medium: 7
Total unique values of column channelGrouping: 8
Total unique values of column deviceCategory: 3
Total unique values of column country: 216


Drop kolom `source` dan `country` di train dan test, karena memiliki kardinalitas yang tinggi.

In [544]:
X_train_2.drop(["source", "country"], axis=1, inplace=True)
X_test_2.drop(["source", "country"], axis=1, inplace=True)

In [545]:
new_cat_columns_2 = ["latest_ecommerce_progress", "bounces", "medium", "channelGrouping", "deviceCategory"]
for cat_col in new_cat_columns_2:
  print(f'All unique values of column {cat_col}: {X_train_2[cat_col].unique()}')

All unique values of column latest_ecommerce_progress: [0 2 3 5 4 6 1]
All unique values of column bounces: [1 0]
All unique values of column medium: ['organic' 'referral' '(none)' 'affiliate' 'cpc' 'cpm' '(not set)']
All unique values of column channelGrouping: ['Organic Search' 'Social' 'Direct' 'Referral' 'Affiliates' 'Paid Search'
 'Display' '(Other)']
All unique values of column deviceCategory: ['desktop' 'mobile' 'tablet']


Ganti nilai `(not set)` di kolom `medium` menjadi `unknown`

In [546]:
X_train_2["medium"] = X_train_2["medium"].str.replace("(not set)", "unknown")
X_test_2["medium"] = X_test_2["medium"].str.replace("(not set)", "unknown")

In [547]:
print(f'All unique values of column medium: {X_train_2["medium"].unique()}')
print(f'All unique values of column medium: {X_test_2["medium"].unique()}')

All unique values of column medium: ['organic' 'referral' '(none)' 'affiliate' 'cpc' 'cpm' 'unknown']
All unique values of column medium: ['(none)' 'organic' 'referral' 'cpc' 'affiliate' 'cpm' 'unknown']


In [548]:
X_train_2.head()

,latest_ecommerce_progress,bounces,time_on_site,pageviews,medium,channelGrouping,deviceCategory
278050,0,1,0,1.0,organic,Organic Search,desktop
387674,0,0,61,2.0,organic,Organic Search,mobile
438340,0,1,0,1.0,referral,Social,desktop
42689,0,1,0,1.0,(none),Direct,desktop
12671,2,0,161,8.0,referral,Referral,desktop


cek skewness di kolom numerikal

In [549]:
for num_col in num_columns_2:
  print(f'Skewness of column {num_col}: {X_train_2[num_col].skew()}')

Skewness of column time_on_site: 6.151275473201027
Skewness of column pageviews: 8.471037650798015


Lakukan capping dengan metode `iqr` untuk kolom `time_on_site` dan `pageviews` karena memiliki nilai skewness yang tinggi. Juga berikan argumen fold = `3`, karena nilai skewness nya tinggi.

In [550]:
winsorizer_2 = Winsorizer(capping_method='iqr',
                             tail='both',
                             fold=3,
                             variables=['time_on_site', 'pageviews'],
                             missing_values='ignore')

X_train_2 = winsorizer_2.fit_transform(X_train_2)
X_test_2 = winsorizer_2.transform(X_test_2)

Nilai skewness sudah berkurang

In [551]:
for num_col in num_columns_2:
  print(f'Skewness of column {num_col}: {X_train_2[num_col].skew()}')

Skewness of column time_on_site: 1.5944783964697067
Skewness of column pageviews: 1.8334069883395525


Check for missing values

In [552]:
print(X_train_2.isna().sum())
print(X_test_2.isna().sum())

latest_ecommerce_progress     0
bounces                       0
time_on_site                  0
pageviews                    10
medium                        0
channelGrouping               0
deviceCategory                0
dtype: int64
latest_ecommerce_progress    0
bounces                      0
time_on_site                 0
pageviews                    4
medium                       0
channelGrouping              0
deviceCategory               0
dtype: int64


In [553]:
scaler_2 = MinMaxScaler()
encoder_2 = OneHotEncoder()
num_imputer = SimpleImputer(strategy='median')

Menggunakan `SimpleImputer` dengan strategy `median` untuk handle missing values

In [554]:
num_pipe_2 = Pipeline([('imputation', num_imputer), ('scaling', scaler_2)])
cat_pipe_2 = Pipeline([('encoding', encoder_2)])

In [555]:
transformer_2 = ColumnTransformer([('numerical', num_pipe_2, num_columns_2),
                                 ('categorical', cat_pipe_2, new_cat_columns_2)], 
                                 remainder='drop')
transformer_2

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling', MinMaxScaler())]),
                                 ['time_on_site', 'pageviews']),
                                ('categorical',
                                 Pipeline(steps=[('encoding',
                                                  OneHotEncoder())]),
                                 ['latest_ecommerce_progress', 'bounces',
                                  'medium', 'channelGrouping',
                                  'deviceCategory'])])

In [556]:
transformer_2.fit(X_train_2)
X_train_final_2 = transformer_2.transform(X_train_2)
X_test_final_2 = transformer_2.transform(X_test_2)

In [557]:
X_train_final_2.toarray()[:5]

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ],
       [0.1882716 , 0.08333333, 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
  

## ii. Model Definition

Menggunakan Logistic Regression, karena model machine learning ini sangat bagus untuk memprediksi kasus klasifikasi.

In [558]:
model_logreg_2 = LogisticRegression(penalty='l2', C=1.0, random_state=0)

In [559]:
lr_pipe_2 = Pipeline([('feature_engineering', transformer_2),
                    ('lr', model_logreg_2)])

lr_pipe_2

Pipeline(steps=[('feature_engineering',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   MinMaxScaler())]),
                                                  ['time_on_site',
                                                   'pageviews']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoding',
                                                                   OneHotEncoder())]),
                                                  ['latest_ecommerce_progress',
                                                   'bounces', 'medium',
                                                   'channelGrouping',
                                                   'deviceCategory'])])),
                ('lr', LogisticRegression(random_state=0))])

In [560]:
X_test_2.head()

,latest_ecommerce_progress,bounces,time_on_site,pageviews,medium,channelGrouping,deviceCategory
200053,0,1,0,1.0,(none),Direct,desktop
389616,0,0,82,6.0,organic,Organic Search,mobile
32958,0,1,0,1.0,referral,Social,desktop
502003,0,1,0,1.0,referral,Social,desktop
160790,2,0,324,13.0,organic,Organic Search,desktop


## iii. Model Training

In [561]:
lr_pipe_2.fit(X_train_2, y_train_2)
y_pred_train_2 = lr_pipe_2.predict(X_train_2)
y_pred_test_2 = lr_pipe_2.predict(X_test_2)
y_pred_train_2

array([0, 0, 0, ..., 0, 0, 0])

## iv. Model Evaluation

In [562]:
y_pred_proba_lr_pipe_2 = lr_pipe_2.predict_proba(X_test_2)[:, 1]
auc_logreg_2 = roc_auc_score(y_test_2, y_pred_proba_lr_pipe_2)

print('AUC Score:', auc_logreg_2)
print('AUC Category:', check_roc_auc_score(auc_logreg_2))

AUC Score: 0.9183861080186331
AUC Category: Good


In [563]:
y_pred_proba_lr_pipe_2 = lr_pipe_2.predict_proba(X_train_2)[:, 1]
auc_logreg_2 = roc_auc_score(y_train_2, y_pred_proba_lr_pipe_2)

print('AUC Score:', auc_logreg_2)
print('AUC Category:', check_roc_auc_score(auc_logreg_2))

AUC Score: 0.919095204253869
AUC Category: Good


Dapat dilihat di atas, bahwa kedua AUC Score (X_test terhadap y_test dan X_train terhadap y_train) memiliki nilai yang tinggi, yaitu 0.9. Hal ini berari model kita good-fit dan dapat memprediksi data dengan baik.

In [564]:
print('Error - Train set:', mean_absolute_error(y_train_2, y_pred_train_2))
print('Error - Test set:', mean_absolute_error(y_test_2, y_pred_test_2))

Error - Train set: 0.015580557959632872
Error - Test set: 0.01510401641183447


Dapat dilihat di atas, bahwa kedua error test nilainya rendah, berarti model kita adalah good-fit.

### 5. Lakukan evaluasi pada model tersebut. Berikan penjelasan apakah model tersebut lebih bagus setelah penambahan fitur ? 

AUC Score pada dataset_1 adalah 0.7766557738985467, yang berarti nilai tersebut termasuk "Decent", sedangkan
AUC Score pada dataset_2 adalah 0.9160851875189232, yang berarti nilai tersebut termasuk "Good".
Berarti model menjadi lebih bagus setelah dilakukan penambahan fitur.

### 6. Jelaskan apakah model tersebut overfit, good-fit atau underfit.

Model memiliki AUC Score yang baik pada train dan test, yaitu lebih dari 0.9, yang mengindikasikan bahwa model tersebut adalah good-fit.

In [565]:
X_train_2["channelGrouping"].value_counts()

channelGrouping
Organic Search    185354
Social            150488
Direct             74224
Referral           39302
Paid Search         9545
Affiliates          7827
Display             1195
(Other)               20
Name: count, dtype: int64

In [566]:
X_train_2["medium"].value_counts()

medium
referral     189790
organic      185354
(none)        74224
cpc            9545
affiliate      7827
cpm            1195
unknown          20
Name: count, dtype: int64

In [567]:
X_train_2["deviceCategory"].value_counts()

deviceCategory
desktop    354124
mobile      99068
tablet      14763
Name: count, dtype: int64

In [568]:
X_train_2["latest_ecommerce_progress"].value_counts()

latest_ecommerce_progress
0    400856
2     40284
3     13657
6      5795
5      5005
4      2249
1       109
Name: count, dtype: int64

### 7. Apakah roc_auc_score merupakan metrik terbaik untuk evaluasi model tersebut atau adakah metrik lain yang cocok ?

Ya, pada data ini, roc_auc_score merupakan metrik terbaik untuk evaluasi model, karena, seperti yang di lihat di atas, banyak data yang imbalance.

### 8. Lakukanlah Hyperparameter Tuning dengan fitur yang telah Anda seleksi dan evaluasi hasilnya. Apakah model anda lebih baik dari model sebelumnnya ?

# ix. Model Saving

In [569]:
with open('model.pkl', 'wb') as file_5:
  pickle.dump(lr_pipe_2, file_5)